# Matplotlib Distribution Plots

There are various statistical methods available to understand how a sequence of data is distributed. The mean, median, variance, minimum, maximum, and various quantiles are all examples of statistics that describe how data is distributed. Often times, the best way to understand how data is distributed is through a visualization. In this chapter, we will visualize distributions of numeric columns of data with matplotlib. Let's begin by reading in a few of the columns of the housing dataset that all have units of square feet.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('mdap.mplstyle')
cols = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
        'GrLivArea', 'GarageArea', 'WoodDeckSF']
housing = pd.read_csv('../data/housing.csv', usecols=cols)
housing.head(3)

The simplest way to get basic non-visual descriptive statistics is to use the `describe` method.

In [ ]:
housing.describe(percentiles=[.01, .1, .25, .5, .75, .9, .99]) \
       .style.format('{:,.0f}')

## Histograms

A histogram is one of the most common methods to visualize the distribution of a single column of numeric data. A histogram is constructed by first binning the numeric data into some number of bins and then counting the values that fall within each bin. Each bin is then plotted as a rectangular bar with width stretching from the left to right edge of the bin and the height equal to the count. Before we create a histogram with matplotlib, let's create the necessary summary statistics with pandas. 

We use the `cut` function to create 10 bins of equal size of the `TotalBsmtSF` column. The bin widths are determined by finding the range of the distribution (maximum minus minimum) and dividing by the number of bins. The `precision` parameter controls the number of decimal places to use for the bins and is set to 0 to return integers. By default, bins include values that equal their right edge exactly and exclude them if they equal their left edge. We set `right` to `False` to reverse this so that it matches how matplotlib does its bin calculations.

In [ ]:
tbsf_bins = pd.cut(housing['TotalBsmtSF'], bins=10, precision=0, right=False)
tbsf_bins.head(3)

We can then call the `value_counts` method on this Series to get the counts within each bin. By default, the returned Series is sorted by count in descending order. With histograms, we want the bins (the index in this case) to be sorted, so we set `sort` to `False`.

In [ ]:
tsbf_bin_count = tbsf_bins.value_counts(sort=False)
tsbf_bin_count

Let's return to matplotlib and use the `hist` axes method to create a histogram of `TotalBsmtSF`. It has a single required parameter, `x`, which may be set as a Series. By default, 10 bins are used.

In [ ]:
fig, ax = plt.subplots()
hist_return = ax.hist(housing['TotalBsmtSF'])

The bar heights and bin widths appear to match the counts from pandas. The `hist` method returns the bin counts (height), bin edges, and the patches used to create the rectangles as a three-item tuple. Let's unpack this into three separate variables.

In [ ]:
counts, edges, patches = hist_return

The edges are contained in a one-dimensional numpy array. There will always be one more edge value than the number of bins. Let's verify that they match the pandas intervals from above.

In [ ]:
edges

Only the last value is different, and this is because pandas always adds 1% to either the first or last bin to include the minimum or maximum value. With matplotlib, the first and last bins have both the left and right edges inclusive. Let's now verify the counts are the same.

In [ ]:
counts

The patch object is a list-like object of all the 10 rectangle patches.

In [ ]:
patches

We can use the `edges` and `counts` arrays to create a line plot connecting the upper left corners of each bar. The extra last edge is not included as there is one more edge than count.

In [ ]:
ax.plot(edges[:-1], counts)
fig

### Customizing the histogram

There are a large number of parameters available to the `hist` method to customize the appearance of the histogram. Our dataset has one extreme value several times larger than the others that's leaving the right side of the plot empty. We can set the `range` parameter to a two-item tuple of the minimum and maximum values to consider. The number of bins is controlled by the `bins` parameter and set to 20.

In [ ]:
fig, ax = plt.subplots()
ax.hist(housing['TotalBsmtSF'], range=(0, 2500), bins=20);

Instead of showing the count of each bin, the bar heights can equal the accumulated total up through that bin by setting the `cumulative` parameter to `True`.

In [ ]:
fig, ax = plt.subplots()
ax.hist(housing['TotalBsmtSF'], range=(0, 2500), bins=20, cumulative=True);

Instead of showing the raw counts, the relative frequency can be shown by setting `density` to `True`. An outline of the heights can be drawn by setting the `histtype` to `'step'`.

In [ ]:
fig, ax = plt.subplots()
ax.hist(housing['TotalBsmtSF'], range=(0, 2500), bins=20, 
        cumulative=True, density=True, histtype='step');

According to the histogram above, around 50% of homes have less than 1,000 total basement square feet. We can verify this with pandas.

In [ ]:
(housing['TotalBsmtSF'] < 1000).mean()

All properties available to matplotlib patches are able to be set within the `hist` method as well. We set the face and edge colors and edge width below.

In [ ]:
fig, ax = plt.subplots()
ax.hist(housing['TotalBsmtSF'], range=(0, 2500), bins=20, 
        fc=(0, .5, 0, .7), ec='black', lw=2);

### Multiple histograms

It's possible to simultaneously plot two or more independent histograms on the same axes. To do so, select the columns you want and retrieve the underlying numpy array with the `values` attribute. The `hist` method doesn't work well when passing it a DataFrame. Here, we select the first and second floor square footage columns and output the values for the first three homes.

In [ ]:
x = housing[['1stFlrSF', '2ndFlrSF']].values
x[:3]

This array is passed as the first argument to `hist`. The `label` parameter works differently than other plotting methods and can be set as a list that corresponds to each column. Each histogram is plotted with bars of the same color corresponding to the color cycle.

In [ ]:
fig, ax = plt.subplots()
ax.hist(x, label=['First Floor', 'Second Floor'], range=(0, 2000))
ax.legend();

The bin edges are computed by first finding the minimum and maximum of the whole dataset regardless of the column.

In [ ]:
housing[['1stFlrSF', '2ndFlrSF']].agg(['min', 'max'])

The minimum of 0 comes from the second floor with the maximum coming from the first floor. The entire range would have been (0, 4,692) if it had not been set explicitly. Each bar is about half of the width of what it would have been if it were the only column plotted. matplotlib automatically decreases the width so that each bar fits within the size of the bin. Let's plot a histogram of just the first floor so a comparison can be made.

In [ ]:
fig, ax = plt.subplots()
ax.hist(housing['1stFlrSF'], range=(0, 2000));

It's possible to plot this single histogram as it appeared in the plot above with both histograms by expanding the y-limit and using `rwidth` (relative width, defaulted to 1) to shrink the width.

In [ ]:
fig, ax = plt.subplots()
ax.hist(housing['1stFlrSF'], range=(0, 2000), rwidth=.4)
ax.set_ylim(0, 850);

The plot with histograms for both variables is difficult to interpret. I prefer changing `histtype` to `'step'`, which will use the entire width of the bin as the width of each horizontal step. We exclude the houses without a second floor below (those with 0 second floor square feet) with the `range` parameter.

In [ ]:
fig, ax = plt.subplots()
ax.hist(x, label=['First Floor', 'Second Floor'], bins=20, range=(100, 2000), 
        histtype='step', alpha=.6, lw=2)
ax.legend();

Histograms for each column can be plotted at once, though each additional column makes the plot considerably more difficult to read. The kernel density estimate plot, which will be introduced in the next part of the book, is a better choices when comparing multiple distributions like this.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ax.hist(housing.values, bins=20,label=housing.columns, range=(100, 3000), 
        histtype='step', alpha=.6, lw=2)
ax.legend();

### Setting the data parameter to a DataFrame

It's possible to set the `data` parameter to a DataFrame and use the column name as a string, but that syntax only works for a single column and cannot be extended to create multiple histograms.

## Box and whisker plots

Box and whisker plots provide a different visual display for analyzing the distribution of a single column of data. A box plot is created by finding the 25th, 50th, and 75th percentiles of the distribution, which are also known as the first quartile, median, and third quartile. Let's do this manually for the `TotalBsmtSF` column.

In [ ]:
quartiles = housing['TotalBsmtSF'].quantile([.25, .5, .75])
quartiles

The **interquartile range** (IQR) is calculated, which is the difference between the third and first quartiles.

In [ ]:
q1 = quartiles.iloc[0]
median = quartiles.iloc[1]
q3 = quartiles.iloc[2]
iqr = q3 - q1
iqr

The lower and upper whiskers are a distance of 1.5 times the IQR beyond their respective quartile.

In [ ]:
lower_whisker = q1 - 1.5 * iqr  
upper_whisker = q3 + 1.5 * iqr
lower_whisker, upper_whisker

When creating a horizontal box and whisker plot, the "box" portion is formed by creating a rectangle that has a left edge at the first quartile and a right edge at the third quartile. A vertical line is placed at the median. The height of the rectangle is not consequential. Horizontal lines are drawn protruding from the left and right edges of the boxes to their respective whisker location. Any individual points outside of the whiskers are called **fliers** and are drawn individually. 

Let's use the `boxplot` method to create a horizontal box, whiskers, and fliers. It has a single required parameter, `x`, the sequence of values, which we set as a Series. By default, the box plot is created vertically, but setting `vert` to `False` makes it horizontal. The `labels` parameter can be set to a list of the column names. For now, we just have one.

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1))
x = housing['TotalBsmtSF']
box_return = ax.boxplot(x, vert=False, labels=['TotalBsmtSF'])

Box plots help identify extreme points better than histograms as we can clearly see exactly where they are located. The returned object is a dictionary containing six different groups of plotting objects. Let's take a look at the different groups by accessing the keys.

In [ ]:
box_return.keys()

Let's retrieve the whiskers and see what type of object they are.

In [ ]:
whiskers = box_return['whiskers']
whiskers

Both whiskers are plotted as a lines from the box edge to the whisker value. Let's verify that the left whisker begins at the third quartile (795.75) and ends at the whisker (42) by getting the x-values.

In [ ]:
whiskers[0].get_xdata()

There's a mismatch between our calculated value of the whisker and the location matplotlib used. This is because no house had total basement square feet between 42 and 105, so matplotlib uses the last known value less than the calculated whisker. 

### More box plot parameters

As usual, there are many different parameters that can be set to control every aspect of the box plot. Here, we use `whis` to set the whiskers at the 5th and 99th percentiles. The width of the box is increased and the box is turned into a patch object. We then change the properties of five distinct plotting elements.

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1))
x = housing.query('TotalBsmtSF < 3_000')['TotalBsmtSF']
ax.boxplot(x, vert=False, labels=['TotalBsmtSF'], 
           whis=(5, 99), widths=.4, patch_artist=True, 
           boxprops={'hatch': 'xox', 'fc': 'plum'},
           flierprops={'marker': '|', 'ms': 15},
           whiskerprops={'lw': 2, 'c': 'crimson', 'ls': '--'},
           medianprops={'lw': 4, 'c': 'blue'},
           capprops={'c': 'crimson', 'lw': 5});

All of the columns can be plotted simultaneously as individual box and whisker plots. By default, each box is placed exactly one y (or x) unit apart.

In [ ]:
fig, ax = plt.subplots()
x = housing.values
ax.boxplot(x, vert=False, labels=housing.columns, widths=.8)
ax.set_xlim(-200, 3_500);

## Exercises